In [24]:
import requests


def parse_name(x):
  url = "http://localhost:30888/lab/entities/parse"
  body = {
    "user_input": x,
    "entities": "Nombre::SegundoNombre::Apellido"
  }
  
  response = requests.post(url, json=body)

  
  if response.status_code == 200:
    parsed_response = response.json()
    print(parsed_response)
    try:
      entities = [parsed_response['Nombre'], parsed_response['SegundoNombre'], parsed_response['Apellido']]
      return entities

    except KeyError:
      print("No se pudo extraer bien el nombre")
      entities = [x,"", ""]
      return entities



In [25]:
def clean_number(x):
    url = "http://localhost:30888/lab/entities/parse"
    body = {"user_input": x, "entities": "NumeroCelular"}

    response = requests.post(url, json=body)
    if response.status_code == 200:
        return response.json()["NumeroCelular"]
    else:
        return None

In [26]:
import pandas as pd
import requests
import re

# Define the functions clean_number(), parse_name(), and create_invite() here

# Read the CSV file
df = pd.read_csv('../data/org6.csv')




In [27]:
# Iterate over each row
for index, row in df.iterrows():
    # print the row number
    print("Row Number: ", index)
    # Clean the phone number
    cleaned_number = clean_number(row["Número de WhatsApp 📞"])

    # Parse the full name
    first_name, second_name, family_name = parse_name(row["Nombre y Apellido 🙌"])

    # Create the invite
    # invite = create_invite(first_name)
    # print(invite)

    # Prepare the data to be posted
    data = {
        "first_name": first_name,
        "second_name": second_name,
        "family_name": family_name,
        "email": row["Dirección de correo electrónico"],
        "whatsapp": cleaned_number,
    }

    # Post the data to the endpoint
    response = requests.post("https://cms.saia.ar/items/leads", json=data)

    # Check the response status
    if response.status_code == 200:
        print(response.json()["data"]["id"])
        print("Data posted successfully")

    else:
        print("Failed to post data")

Row Number:  0
{'Nombre': 'Silvia', 'SegundoNombre': '', 'Apellido': 'García'}
1848
Data posted successfully
Row Number:  1
{'Nombre': 'Claudio', 'SegundoNombre': '', 'Apellido': 'Crescentini'}
1849
Data posted successfully
Row Number:  2
{'Nombre': 'Mateo', 'SegundoNombre': '', 'Apellido': 'Pedrozo'}
1850
Data posted successfully
Row Number:  3
{'Nombre': 'Ezequiel', 'SegundoNombre': '', 'Apellido': 'Ciciliani'}
1851
Data posted successfully
Row Number:  4
{'Nombre': 'Mauro', 'SegundoNombre': 'Cayetano', 'Apellido': 'Vera'}
1852
Data posted successfully
Row Number:  5
{'Nombre': 'Mauro', 'SegundoNombre': 'Cayetano', 'Apellido': 'Vera'}
Failed to post data
Row Number:  6
{'Nombre': 'María', 'SegundoNombre': 'Marcela', 'Apellido': 'Balzarelli'}
1854
Data posted successfully
Row Number:  7
{'Nombre': 'Daniela', 'SegundoNombre': '', 'Apellido': 'Mandaglio'}
1855
Data posted successfully
Row Number:  8
{'Nombre': 'Paula', 'SegundoNombre': '', 'Apellido': 'Yedro'}
1856
Data posted successfu

In [28]:
import requests

for index, row in df.iterrows():
    email = row["Dirección de correo electrónico"]
    url = f'https://cms.saia.ar/items/leads?filter={{"email":{{"_eq":"{email}"}}}}'

    # Get the ID
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data and "data" in data and len(data["data"]) > 0:
            contact_id = data["data"][0]["id"]
        else:
            print(f"No contact found for email: {email}")
            continue
    else:
        print(f"Failed to get contact for email: {email}")
        continue

    # Patch the contact
    patch_url = f"https://cms.saia.ar/items/leads/{contact_id}"
    knows_saia_from = row["¿Cómo te enteraste de SAIA? 🤔"]
    saiaconf_stream = (
        row["¿Cómo vas a participar de SAIAConf? 👇"
       ] == "Streaming (Quiero recibir link para sumarme al evento)"
    )

    body = {
        "saiaconf_ad": False,
        "knows_saia_from": knows_saia_from,
        "saiaconf_stream": saiaconf_stream,
    }

    response = requests.patch(patch_url, json=body)
    if response.status_code == 200:
        print(f"Contact patched successfully for email: {email}")
    else:
        print(f"Failed to patch contact for email: {email}")

Contact patched successfully for email: garciadecajen@gmail.com
Contact patched successfully for email: ccrescentini@frba.utn.edu.ar
Contact patched successfully for email: mateopedrozo036@gmail.com
Contact patched successfully for email: eciciliani@hotmail.com
Contact patched successfully for email: mauro0129arg@gmail.com
Contact patched successfully for email: mauro0129arg@gmail.com
Contact patched successfully for email: marcebalzarelli@gmail.com
Contact patched successfully for email: Lic.mandaglio.ux@gmail.com
Contact patched successfully for email: pawyedro@gmail.com
Contact patched successfully for email: danielanmandaglio@gmail.com
Contact patched successfully for email: amorgavi@qactions.com
No contact found for email: ezequielbz@outlook.es
Contact patched successfully for email: julietalopezivern@gmail.com
Contact patched successfully for email: ferav234@gmail.com
Contact patched successfully for email: bautisosaiphone7@gmail.com
Contact patched successfully for email: ramiro